In [1]:
# ----------------------------------------------------------------------------------------------
# MAIS202 Group 1 - Fall 2022 Final Project
# Jennifer Tram Su, Lucy Mao
# Image Colourization
#
# Input : Single channel representation of greyscale image
# Output : Two/three channel representation of colourized image
#
# References : 
# @Article{MaskedAutoencoders2021,
#   author  = {Kaiming He and Xinlei Chen and Saining Xie and Yanghao Li and Piotr Doll{\'a}r and Ross Girshick},
#   journal = {arXiv:2111.06377},
#   title   = {Masked Autoencoders Are Scalable Vision Learners},
#   year    = {2021},
# }
# ----------------------------------------------------------------------------------------------

# File Handling
import glob

# Manipulation
import numpy as np
from PIL import Image

# Models
import tensorflow as tf
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator


2022-11-15 09:19:42.439839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 09:19:42.980846: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-15 09:19:43.061086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-15 09:19:43.061111: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [8]:
# ----------------------------------------------------------------------------------------------
# Train Test Split - Update with new data

path = 'Datasets'
ab = path + '/ab/*'
l = path + '/l/*' 

lfiles = glob.glob(l)
abfiles = glob.glob(ab)

abs = [np.load(abfiles[i]) for i in range(len(abfiles))] # [num photos, dim1, dim2, 2]
ls = np.load(lfiles[0])



In [10]:
#ab_train = np.append(abs[1], abs[2], axis=0) # abs[0], abs[1]
ab_train = abs[1]
ab_test = abs[0]

l_train = ls[10000:20000, :, :]
l_train = l_train.reshape((10000, 224, 224, 1))
l_test = ls[0:5000, :, :]

(10000, 224, 224, 1)


In [11]:
X = l_train
y = ab_train

#Encoder
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(224, 224, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

#Decoder
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()

#history = model.fit(x=X,y=y,validation_split=0.2, epochs=10, batch_size=32, verbose=0)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      640       
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 56, 56, 128)       147584    
                                                                 
 conv2d_4 (Conv2D)           (None, 56, 56, 256)       295168    
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 256)       590080    
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 512)       1180160   
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 512)      

In [ ]:
out = model.predict(l_test)

In [ ]:
out = out*128

l = l_test[0]
a = out[0, :, :, 0]
print(a)

In [ ]:
mkdir models

In [ ]:
model.save('model/test')